# Polymer dropcast experiment setup



#### Necessary Imports 

As usual, let's start by importing the necessary packages we will need to run our demo.

In this case, we will also set up our matplotlib- the package we will use to visualize the beautiful art that our Rob Ross will generate as an external window.

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import Pipette

# ------ Data visualization outside of the notebook ------
import matplotlib
import pandas as pd

### Jubilee platfform connection and set-up

In [3]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False) 

Before we can set up the `deck` of our `Jubilee` with all our labware, we will need to *home* **all** its axis.

> Attention -- !!! **BEFORE** !!! running the next cell, make sure that NO labware is loaded onto the deck or the Tool Changer will crush into them. 

In [4]:
jubilee.home_all()

### Deck preparation and labware definition 
Awesome, now that we have our machine all ready to go, we will need to load all the labware onto it both *physically* and *digitally* . 

* Physically: place the desired/required labware for the Demo onto the deck of your Jubilee
* Digitally: first, we need to define our `deck`; then define and load the `labware` we want to use and indicate on which `slot` each of them was assigned to

In [5]:
# ---------------- Deck -------------------
deck = jubilee.load_deck('lab_automation_deck.json')

To run this *autonoous* color matching demo, we will need at least four types of labware:

* A **Tiprack** that is suitable for the pipette you are using to transfer your liquids
* A **Sample Labware** that will be used to collect the colors that your Rob Ross will make 
* A **Stock Labware** that will host the stock colors that your Rob Ross can mix together
* A **trash** to collect the used tips

In [6]:
deck.safe_z

10

In [7]:
# -------------- Labware ------------------
tiprack = jubilee.load_labware('opentrons_96_tiprack_300ul.json', 0)
tiprack.load_manualOffset()
sample_wellplate = jubilee.load_labware('fisherbrand_96_wellplate_360ul.json', 2) #Replace this with custom labware for samples
#samples.load_manualOffset()
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 3)
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

New manual offset applied to opentrons_96_tiprack_300ul


In [8]:
#UL = (26.5, 174.0)
#UR = (126.6, 173.4)
#BR = (126.5, 110.9)
#samples.manual_offset((UL, UR, BR))

UL = (29.4, 82.1)
UR = (127, 82.1)
BR = (127, 19.2)
tiprack.manual_offset((UL, UR, BR))


UL = (174.5, 171.0)
UR = (260.5, 170.0)
BR = (260.5, 114.0)
stocks.manual_offset((UL, UR, BR))

New manual offset applied to opentrons_96_tiprack_300ul
New manual offset applied to 20mlscintillation_12_wellplate_18000ul


You can notice that the *tiprack* and the *samples* labware have a `manualOffset()` applied to them. This is to ensure that the labware coordinates are as precise as we can. The manual offset is **labware** and **slot** dependent, so you will need to run it prior to do so. You can find more information on how to run this in the `Science Jubilee` repository/documentation

This next 

In [9]:
# ------------- Stocks ------------
stock_1 = stocks[0].bottom(+5)
stock_2 = stocks[1].bottom(+5)



In [10]:
stock_1

Location(point=(174.5, 171.0, 12.5), labware=Well A1 form 20mLscintillation 12 Well Plate 18000 µL on slot 3)

In [11]:
tiprack.manualOffset

{'0': ((29.4, 82.1), (127, 82.1), (127, 19.2))}

### Tool import and set up

Once again we will need to load all our tools onto our machine both *physically* and *digitally* . 

To learn how to load the tools *physically*, you can find more information and resources in the `Science_Jubilee` [documentation](https://machineagency.github.io/science_jubilee/building/building_tools.html#building-tools)

Each tool will at minimum require a `tool_index`, an integer, and a `tool_name`. These should be the same as the ones defined in the machine's `config.g` file. 
Some tools will also required further information. You can defined these individually or in a config file for easier tool load. 

**Pipette Tool**

In [12]:
P300 = Pipette.Pipette.from_config(1, 'Pipette', 'P300_config.json')
jubilee.load_tool(P300)

Even though you will be able to use te pipette after these two single lines, you can also associate its `tiprack` to the tool, as well as define a `trash` location. This will just make the code more clean.

In [13]:
P300.add_tiprack(tiprack)
P300.trash = trash[0]

## Load experiment definition files

In [14]:
experiment_def = pd.read_csv('experiment_definition_draft.csv')

In [15]:
experiment_def

,Sample UUID,Sample Name,Stock Solution UUID,Volume dispensed [uL],"Dispense offset from center [X, Y] [mm]",Dispense rate [uL/S]
0,1,test_sample,1234,200,"(2,2)",20


In [16]:
### write wrapper code to get sample values extracted from dataframe

In [17]:
stock_location = stock_1
volume = 200
dispense_offset = (2,2)
dispense_rate = 20

sample_1 = {'stock':stock_1, 'volume':volume, 'dispense_offset':dispense_offset, 'dispense_rate':dispense_rate}

In [18]:
samples = [sample_1]

for i, sample in enumerate(samples):
    sample['location'] = sample_wellplate[i]
    
    

## Run sample preparation experiment

In [19]:
samples

[{'stock': Location(point=(174.5, 171.0, 12.5), labware=Well A1 form 20mLscintillation 12 Well Plate 18000 µL on slot 3),
  'volume': 200,
  'dispense_offset': (2, 2),
  'dispense_rate': 20,
  'location': Well A1 form Fisherbrand 96 Well Plate 360 ÂµL on slot 2}]

In [20]:
loc = samples[0]['location']

In [21]:
loc.x

28.68

In [22]:

jubilee.pickup_tool(P300)
for sample in samples:
    stock = sample['stock']
    volume = sample['volume']
    dispense_offset = sample['dispense_offset']
    dispense_rate = sample['dispense_rate']
    location = sample['location']

    location.x = location.x + dispense_offset[0]
    location.y = location.y + dispense_offset[1]
    location.bottom(+20)

    #convert dispense rate into mL/minute
    

    P300.pickup_tip()
    P300.aspirate(volume, stock)

    P300.dispense(volume, stock, s = dispense_rate)
    P300.drop_tip()
    

ValueError: Location should be of type Well or Tuple